# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

In [55]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [56]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()

# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Bind the 'class' (wine target) to the DataFrame
wine_df['class'] = wine_data.target

# Display the DataFrame
wine_df


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [57]:
# Your answer here: 178 entries
num_observations = wine_df.shape[0]
print(f"Number of observations: {num_observations}")

Number of observations: 178


_(ii)_ How many variables (columns) does the dataset contain?

In [58]:
# Your answer here: 14 columns
num_columns = wine_df.shape[1]
print(f"Number of columns: {num_columns}")

Number of columns: 14


_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [59]:
# Your answer here: Categorical. Unique values are: 0, 1, 2
print("unique values: " + str(wine_df.groupby("class").size()))

unique values: class
0    59
1    71
2    48
dtype: int64



_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [60]:
# Your answer here: 13 variables
print("Total number of variables:", len(wine_df.describe().drop(columns = 'class').columns))

Total number of variables: 13


You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [61]:
# Select predictors (excluding the last column)
predictors = wine_df.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
0  1.518613   -0.562250  0.232053          -1.169593   1.913905   
1  0.246290   -0.499413 -0.827996          -2.490847   0.018145   
2  0.196879    0.021231  1.109334          -0.268738   0.088358   
3  1.691550   -0.346811  0.487926          -0.809251   0.930918   
4  0.295700    0.227694  1.840403           0.451946   1.281985   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0       0.808997    1.034819             -0.659563         1.224884   
1       0.568648    0.733629             -0.820719        -0.544721   
2       0.808997    1.215533             -0.498407         2.135968   
3       2.491446    1.466525             -0.981875         1.032155   
4       0.808997    0.663351              0.226796         0.401404   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.251717  0.362177                      1.847920  1.013009  
1        -0.293321  0.406051

(i) Why is it important to standardize the predictor variables?

> Your answer here...
Standardizing predictor variables is really important because it makes sure that all features contribute equally to the model. Transforming them to have the same scale eliminates the risk of specific features with larger ranges dominating others, which avoids skewing the model. 

Also, standardization helps models converge faster. It improves the performance of distance-based algorithms like k-nearest neighbors (KNN) as these methods rely on distance metrics that can be affected by scale. By making sure that no single feature dominates, standardization allows fair weighting of all variables, resulting in a more reliable and accurate model.

Source: https://www.markovml.com/blog/normalization-in-machine-learning

(ii) Why did we elect not to standard our response variable `Class`?

> Your answer here...
The variable Class has three possible values: 0, 1, and 2, which represent different class labels. It's important to keep these values in their original form because standardizing them could make it hard to understand what they really mean. Since these numbers represent distinct categories, preserving their values ensures the correct interpretation.

(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

> Your answer here...
The seed is important because it gives the same results each time the random operation run. This means it allows reproducing experiments, leading to consistent outcomes every time, helping debug, collaborate with others, and verify findings. 
Also, using the same seed makes it easier to compare different experiments and methods because starting from the same point ensures working with the same data splits.
The specific value of the seed is not important. It is only the starting point for generating random numbers. As long as the same seed is used, the random values produced will be the same

source: https://medium.com/@abhishekjainindore24/seeding-success-a-guide-to-setting-seeds-in-data-science-5d24e00c3dc7


(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [62]:
# Do not touch
np.random.seed(123)
# Create a random vector of True and False values to split the data
split = np.random.choice([True, False], size=len(predictors_standardized), replace=True, p=[0.75, 0.25])

X_train = predictors_standardized[split == True]
X_test = predictors_standardized[split == False]

y_train = wine_df.loc[split, wine_df.columns[-1]]
y_test = wine_df.loc[~split, wine_df.columns[-1]]


#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [63]:
# Your code here...
# Initialize the KNN classifier using `KNeighborsClassifier()`
knn = KNeighborsClassifier()
knn

KNeighborsClassifier()

In [64]:


# Define a parameter grid for `n_neighbors` ranging from 1 to 50.
parameter_grid = {
    "n_neighbors": range(1,51)
}

#Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
wine_tune_grid = GridSearchCV(
    estimator = knn,
    param_grid=parameter_grid,
    cv=10
)

#After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.
wine_tune_grid.fit(
    X_train,
    y_train
)

accurancies_grid = pd.DataFrame(wine_tune_grid.cv_results_)

accurancies_grid

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001323,0.000425,0.002036,0.000507,1,{'n_neighbors': 1},0.857143,0.928571,0.928571,0.928571,1.0,0.928571,1.0,1.000000,0.857143,0.928571,0.935714,0.050000,41
1,0.000953,0.000008,0.001635,0.000009,2,{'n_neighbors': 2},0.928571,0.928571,0.928571,1.000000,1.0,1.000000,1.0,1.000000,0.857143,0.928571,0.957143,0.047380,13
2,0.000971,0.000036,0.001675,0.000049,3,{'n_neighbors': 3},0.928571,1.000000,0.928571,0.928571,1.0,1.000000,1.0,0.928571,0.857143,0.928571,0.950000,0.045737,19
3,0.000983,0.000058,0.001672,0.000052,4,{'n_neighbors': 4},0.928571,0.928571,0.928571,1.000000,1.0,0.928571,1.0,1.000000,0.857143,0.928571,0.950000,0.045737,19
4,0.000967,0.000038,0.001671,0.000051,5,{'n_neighbors': 5},0.928571,0.928571,0.928571,1.000000,1.0,0.928571,1.0,1.000000,0.928571,1.000000,0.964286,0.035714,5
5,0.000966,0.000013,0.001661,0.000025,6,{'n_neighbors': 6},0.928571,0.928571,0.928571,0.928571,1.0,0.928571,1.0,1.000000,1.000000,1.000000,0.964286,0.035714,5
6,0.000952,0.000013,0.001656,0.000052,7,{'n_neighbors': 7},1.000000,0.928571,0.928571,1.000000,1.0,0.928571,1.0,1.000000,0.928571,1.000000,0.971429,0.034993,2
7,0.000961,0.000023,0.001676,0.000072,8,{'n_neighbors': 8},1.000000,0.928571,0.928571,1.000000,1.0,0.928571,1.0,1.000000,1.000000,1.000000,0.978571,0.032733,1
8,0.000965,0.000020,0.001679,0.000047,9,{'n_neighbors': 9},1.000000,0.928571,0.928571,0.928571,1.0,0.928571,1.0,1.000000,0.928571,1.000000,0.964286,0.035714,5
9,0.000957,0.000014,0.001666,0.000021,10,{'n_neighbors': 10},1.000000,0.928571,0.928571,0.928571,1.0,0.928571,1.0,1.000000,1.000000,1.000000,0.971429,0.034993,2


In [65]:
best_param = wine_tune_grid.best_params_["n_neighbors"]
best_param

8

#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [66]:
# Your code here...
knn = KNeighborsClassifier(n_neighbors=best_param)
knn

KNeighborsClassifier(n_neighbors=8)

In [67]:
# Fit the model on the training data
knn.fit(X_train, y_train)

#Evaluate its performance on the test set using `accuracy_score`.

y_pred = knn.predict(X_test)


In [68]:

test_accurancy = accuracy_score(y_test, y_pred )

test_accurancy

0.9473684210526315

# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `11:59 PM - 01/12/2025`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/LCR/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-4-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
